In [1]:
import sys

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

print(sys.executable)

/home/mehrzad/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/mehrzad/anaconda3/envs/dbpedia/bin/python


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

device

'cuda:0'

#
# CodeGen25-7b-instruct

In [2]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen25-7b-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen25-7b-instruct")

# text = "def hello_world():"
# input_ids = tokenizer(text, return_tensors="pt").input_ids
# generated_ids = model.generate(input_ids, max_length=128)
# print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))


A new version of the following files was downloaded from https://huggingface.co/Salesforce/codegen25-7b-instruct:
- tokenization_codegen25.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit mu

/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.75s/it]


def hello_world():
    return 'Hello World!'

@app.route('/api/v1/<string:name>')
def hello_name(name):
    return 'Hello {}!'.format(name)

@app.route('/api/v1/users', methods=['POST'])
def create_user():
    request_data = request.get_json()
    name = request_data['name']
    email = request_data['email']
    return 'User {} with email {} created successfully'.format(name, email)


In [3]:
checkpoint="Salesforce/codegen25-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_4bit=True, device_map="auto")


text = "write a SPARQL query for capital of Japan:"
input_ids = tokenizer(text, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Loading checkpoint shards: 100%|████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.07s/it]
/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Lin

write a SPARQL query for capital of Japan:
SELECT ?capital
WHERE {
  ?country a dcterms:Country .
  ?country rdfs:label ?name .
  ?country rdfs:isPartOf <http://www.wikidata.org/entity/Q17> .
  ?country rdfs:label "Japan"@en .
  ?country cap:capital ?capital .
}
```
This query uses the `cap:capital` property to query for the capital of Japan. The `WHERE` clause contains a nested `


#
# Raw model comparisons 

### Preparing comparison data 
**qald_9_plus: ** https://github.com/KGQA/QALD_9_plus/tree/main

In [81]:
import json

with open('qald_9_plus_test_dbpedia.json', 'r') as f:
    qald_9_plus_data = json.load(f)
    
qald_9_plus_data['questions'][0]

{'id': '99',
 'question': [{'language': 'en',
   'string': 'What is the time zone of Salt Lake City?'},
  {'language': 'de', 'string': 'In welcher Zeitzone liegt Salt Lake City?'},
  {'language': 'de', 'string': 'Was ist die Zeitzone von Salt Lake City?'},
  {'language': 'ru', 'string': 'Какой часовой пояс в Солт-Лейк-Сити'},
  {'language': 'ru',
   'string': 'В каком часовом поясе расположен Солт-Лейк-Сити?'},
  {'language': 'uk', 'string': 'Який часовий пояс у Солт-Лейк Сіті?'},
  {'language': 'lt', 'string': 'Kokia Solt Leik Sičio laiko zona?'},
  {'language': 'be', 'string': 'Які гадзінны пояс у Солт-Лэйк-Сіці'},
  {'language': 'lt', 'string': 'Kokia laiko juosta yra Solt Leik Sityjes'},
  {'language': 'ba', 'string': 'Ниндей вакыт поясы Солт-Лейк-Ситила'},
  {'language': 'es',
   'string': '¿En qué zona horaria se encuentra Salt Lake City?',
   'keywords': 'Salt Lake City,  zona horaria'}],
 'query': {'sparql': 'PREFIX res: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedi

In [82]:
# extracting pairs of question/sparql    
    
english_questions, sparql_queries =[],[]
for data in qald_9_plus_data['questions']:
    english_questions.append(
        next((item['string'] for item in data['question'] if item['language'] == 'en'), None)
    )
    sparql_queries.append(data['query']['sparql'])
    

print(f'{len(english_questions)} pairs extracted from test set: ', "\n")
print(english_questions[:5])
print("\n")
print(sparql_queries[:5])

150 pairs extracted from test set:  

['What is the time zone of Salt Lake City?', 'Who killed Caesar?', 'What is the highest mountain in Germany?', 'Which American presidents were in office during the Vietnam War?', 'Butch Otter is the governor of which U.S. state?']


['PREFIX res: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedia.org/property/> SELECT DISTINCT ?uri WHERE { res:Salt_Lake_City <http://dbpedia.org/ontology/timeZone> ?uri }', 'PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?uri dct:subject dbc:Assassins_of_Julius_Caesar }', 'PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: <http://dbpedia.org/ontology/> SELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation ?elevation ; onto:locatedInArea <http://dbpedia.org/resource/Germany> } ORDER BY DESC(?elevation) LIMIT 1', 'PREFIX dbo: <http://dbpedia.org/ontology/

In [2]:
## TODO
# use huggingface "Evaluate" libray 

from datasets import load_metric


def calculate_bleu(actual, generated):
    bleu_metric = load_metric('bleu')
    bleu_score = bleu_metric.compute(predictions=generated, references=[[s] for s in actual])
    return bleu_score



actual_queries = [
    "SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }",
    "SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }"
]


generated_queries = [
    "SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }",
    "SELECT?capital WHERE { dbr:Japan dbo:capital ?capital . }"
]


# Tokenizing by space; we don't care about the number of spaces if they are more than one
actual_tokenized = [sentence.split() for sentence in actual_queries]
generated_tokenized = [sentence.split() for sentence in generated_queries]


bleu_score = calculate_bleu(actual_tokenized, generated_tokenized)


print(bleu_score['bleu'])

/tmp/ipykernel_34708/2775063547.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')


0.8736581394767808


In [ ]:
## TODO 
# debug the script  

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric
import pandas as pd
import torch

def read_dataset(path):
    dataset = pd.read_csv(path)
    return dataset

def generate_sparql(model, tokenizer, question):
    inputs = tokenizer.encode(question, return_tensors="pt").to(model.device)
    outputs = model.generate(inputs, max_length=100, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0])

def calculate_bleu(actual, generated):
    bleu_metric = load_metric('bleu')
    bleu_score = bleu_metric.compute(predictions=[generated.split()], references=[[actual.split()]])
    return bleu_score['bleu']

def main():
    models = [
        "Salesforce/codegen-350M-multi", "Salesforce/codegen25-7b-instruct", 
        "bigcode/starcoderbase-3b", "bigcode/starcoderbase-1b",
        "EleutherAI/gpt-neo-2.7B", "tiiuae/falcon-rw-1b"
              
            ]  

    # this could be a path to your dataset or a DataFrame containing your dataset
    dataset_path = "/home/mehrzad/repos/mehrz/dbpedia/data/prompt_tuning/nspm_100k_test.csv"

    dataset = read_dataset(dataset_path)

    for model_name in models:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        # if you have a GPU, use it, otherwise use CPU
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        model.to(device)

        results = []
        total_bleu_score = 0

        for idx, row in dataset.iterrows():
            question, actual_sparql = row['question'], row['sparql_query']
            generated_sparql = generate_sparql(model, tokenizer, question)

            # calculate the BLEU score for this question
            bleu_score = calculate_bleu(actual_sparql, generated_sparql)

            # store the BLEU score, actual and generated SPARQL for each question for this model
            results.append((question, actual_sparql, generated_sparql, bleu_score))

            total_bleu_score += bleu_score

        average_bleu_score = total_bleu_score / len(dataset)

        # print or save the results and average BLEU score for this model
        print(f"Results for {model_name}:\n", results)
        print(f"Average BLEU score for {model_name}: ", average_bleu_score)

        # delete the model and tokenizer to free up memory
        del model
        del tokenizer

        # clear the GPU memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

if __name__ == "__main__":
    main()


In [56]:
from datasets import list_datasets

datasets_list = list_datasets()

for dataset in datasets_list:
    if 'sparql' in dataset.lower():
        print(dataset)


print('='*30)

for dataset in datasets_list:
    if 'qald' in dataset.lower():
        print(dataset)

yazdipour/text-to-sparql-kdwd
mshahzaib777/text-sparql
htriedman/wiki-sparql
neo-nlp-dev/qald9


In [57]:
from datasets import load_dataset

dataset = load_dataset('neo-nlp-dev/qald9')

print(dataset['train'][90])

Found cached dataset json (/home/mehrzad/.cache/huggingface/datasets/neo-nlp-dev___json/neo-nlp-dev--qald9-fdb5c4b0c6b3d61f/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 550.69it/s]

{'question': 'Give me all Swedish holidays.', 'tokens': '<var1> <tok> type <tok> holiday <dot> <var1> <tok> country <tok> Sweden'}


In [42]:
from datasets import load_dataset

dataset = load_dataset('yazdipour/text-to-sparql-kdwd')


Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 367.92it/s]


DatasetGenerationError: An error occurred while generating the dataset

In [55]:
from datasets import load_dataset

dataset = load_dataset('mshahzaib777/text-sparql')
print(dataset['test'][0])

Found cached dataset csv (/home/mehrzad/.cache/huggingface/datasets/mshahzaib777___csv/mshahzaib777--text-sparql-e5154e9fc32d0c92/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 278.12it/s]

{'src': 'what is the common military branch of the edwin d . morgan and that of eli lilly', 'trg': 'SELECT DISTINCT var_uri where brack_open dbr_Edwin_D._Morgan dbo_militaryBranch var_uri sep_dot dbr_Eli_Lilly dbp_branch var_uri brack_close'}


#
# StarCoder 3B

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


checkpoint = "bigcode/starcoderbase-3b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:10<00:00,  5.35s/it]


In [7]:
prompt = """
Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX dbc: <http://dbpedia.org/resource/Category:><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= Who killed Caesar?
"answer"=
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX dbc: <http://dbpedia.org/resource/Category:><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }<|endoftext|>

"question"= Who killed Caesar?
"answer"=
SELECT DISTINCT?person?cause
WHERE {
 ?person dbo:died?cause.
 ?cause dbo:cause?kill.
 ?kill dbo:killer?person.
 ?kill dbo:killer?killer.
 ?killer dbo:name "Caesar".
}
<|endoftext|>


In [6]:
prompt = """
# using RDFlib to write a SPARQL query over DBpedia
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# using RDFlib to write a SPARQL query over DBpedia

# +
# Import the RDFlib library
import rdflib

# Import the SPARQLWrapper library
from SPARQLWrapper import SPARQLWrapper, JSON

# Import the DBpedia library
from DBpedia import DBpedia

# +
# Create a DBpedia object
dbpedia = DBpedia()

# Create a SPARQLWrapper object
sparql = SPARQLWrapper("http://dbpedia.org/sparql")



In [83]:
prompt = """
Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX dbc: <http://dbpedia.org/resource/Category:><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= Who killed Caesar?'
"answer"=
"""


sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

NameError: name 'pipeline' is not defined

In [11]:
prompt = """
Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

===

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }

===

"question"= what is the capital of USA?
"answer"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }

===

"question"= what is the capital of Italy?
"answer"= 

"""


sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Result: 
Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

===

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }

===

"question"= what is the capital of USA?
"answer"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }

===

"question"= what is the capital of Italy?
"answer"= 


===

"question"= What are the 3 most populous cities in Europe?
"answer"= SELECT?city?total WHERE {?city dbr:population?total. FILTER (?total > 10000 ) }  ORDER BY DESC(?total) LIMIT 3


===

"question"= What are the 5 most populous cities in the World?
"answer"= SELECT?city?total WHERE {?city dbr:population?total } ORDER BY DESC(?total) LIMIT 5


===

"question"=  What country has the largest population? 
"answer"= SELECT?country WHERE {?country dbr:population?total. FILTER (?total > 140000000

In [15]:
prompt = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }



"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }



"question"= what is the capital of Italy?
"SPARQL"= 

"""


sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Result: 
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }



"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }



"question"= what is the capital of Italy?
"SPARQL"= 


"question"= what is the population of the US state of California?
"SPARQL "SELECT?population WHERE {?state a dbo:StateOfTheUnitedStates ; dbo:capital?capital ; dbo:population?population.?state dbr:California?capital }





"question"= what is the population of the Italy state of Puglia?
"SPARQL "SELECT?population WHERE {?state a dbo:StateOfTheRepublic ; dbo:capital?capital ; dbo:population?population.?state dbr:Puglia?capital }

"question"= what is the population of the USA state of Illinois?

"SPARQL"= SELECT?population WH

In [15]:
prompt = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }



"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }



"question"= what is the capital of Italy?
"SPARQL"= 

"""


sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Result: 
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }



"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }



"question"= what is the capital of Italy?
"SPARQL"= 


"question"= what is the population of the US state of California?
"SPARQL "SELECT?population WHERE {?state a dbo:StateOfTheUnitedStates ; dbo:capital?capital ; dbo:population?population.?state dbr:California?capital }





"question"= what is the population of the Italy state of Puglia?
"SPARQL "SELECT?population WHERE {?state a dbo:StateOfTheRepublic ; dbo:capital?capital ; dbo:population?population.?state dbr:Puglia?capital }

"question"= what is the population of the USA state of Illinois?

"SPARQL"= SELECT?population WH

In [14]:
prompt = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }



"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }



"question"= what is the capital of Italy?
"SPARQL"= SELECT

"""


inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }



"question"= what is the capital of USA?
"SPARQL"= SELECT?capital WHERE { dbr:USA dbo:capital?capital. }



"question"= what is the capital of Italy?
"SPARQL"= SELECT


"question"= what is the capital of France?
"SPARQL"= SELECT


"question"= what is the capital of Germany?
"SPARQL"= SELECT


"question"= what is the capital of


In [19]:
prompt = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }



"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }



"question"= what is the capital of Italy?


"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=50, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>



"question"= what is the capital of Japan?
"SPARQL"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }



"question"= what is the capital of USA?
"SPARQL"= SELECT?capital WHERE { dbr:USA dbo:capital?capital. }



"question"= what is the capital of Italy?



"question"= what is the capital of France?



"question"= what is the capital of Germany?



"question"= what is the capital of Spain?



"question"= what is the capital of


In [5]:
print(tokenizer.eos_token)

<|endoftext|>


To check where end of sequence is detected, let's take a look at how the prompt is encoded

In [31]:
inputs['input_ids'][0]

tensor([  203,   982,    87,  1111,   313, 35462,  2150,    20,   810,  1630,
        27055,   313,  4594,    20,  1991,   312,   358,  4186,  2150,  2467,
         1770,   365,  4581, 35738, 34066,  4794,    44,   203,  9857,   343,
          839,    44,   333,   525,   555,  1199, 35738,    32,  1088,    33,
         3142,  2039,   203,  9857, 36855,    44,   333,   525,   555,  1199,
        35738,    32,  1088,    33, 30142,  2039,  2831,   203,    20,  4594,
           20,    47,  2769,   438,   322, 18926,   432,   971, 23571,    49,
          203,    20, 35462,  2150,    20,    47, 10246,  1018, 21147,  4998,
          301,   343,   839,    44,    60, 23571, 36855,    44, 21147,  1018,
        21147,   638,   320,  2831,   203,    20,  4594,    20,    47,  2769,
          438,   322, 18926,   432, 15070,    49,   203,    20, 35462,  2150,
           20,    47, 10246,  1018, 21147,  4998,   301,   343,   839,    44,
        32713, 36855,    44, 21147,  1018, 21147,   638,   320, 

In [32]:
tokenizer.decode(inputs['input_ids'][0])

'\nCompelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:\nPREFIX dbr: <http://dbpedia.org/resource/>\nPREFIX dbo: <http://dbpedia.org/ontology/>\n\n\n\n"question"= what is the capital of Japan?\n"SPARQL"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }\n\n\n\n"question"= what is the capital of USA?\n"SPARQL"= SELECT?capital WHERE { dbr:USA dbo:capital?capital. }\n\n\n\n"question"= what is the capital of Italy?\n\n\n'

In [15]:
eos_token = tokenizer.eos_token
eos_token_id = tokenizer.eos_token_id

print(f'End of Sequence token:\t{eos_token}\t with id={eos_token_id}\n')

question = "What is the capital of France?\n"
sparql_query = "SELECT ?capital WHERE { dbr:France dbo:capital ?capital .}"

# check if the tokenizer automatically adds the necessary special tokens and attention mask.
inputs = tokenizer.encode_plus(question + sparql_query, return_tensors="pt")
print(inputs)

if eos_token_id not in inputs['input_ids'][0]:
    print(f'\nNo EOS token found in tokenized input')

End of Sequence token:	<|endoftext|>	 with id=0

{'input_ids': tensor([[ 8197,   438,   322, 18926,   432, 45600,    49,   203,  4620,  1018,
         21147,  4998,   301,   343,   839,    44,  6082,   723, 36855,    44,
         21147,  1018, 21147,   638,   111]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

No EOS token found in tokenized input


In [39]:
# encode prompt with encode_plus

inputs = tokenizer.encode_plus(prompt, return_tensors="pt")
inputs

{'input_ids': tensor([[  203,   982,    87,  1111,   313, 35462,  2150,    20,   810,  1630,
         27055,   313,  4594,    20,  1991,   312,   358,  4186,  2150,  2467,
          1770,   365,  4581, 35738, 34066,  4794,    44,   203,  9857,   343,
           839,    44,   333,   525,   555,  1199, 35738,    32,  1088,    33,
          3142,  2039,   203,  9857, 36855,    44,   333,   525,   555,  1199,
         35738,    32,  1088,    33, 30142,  2039,  2831,   203,    20,  4594,
            20,    47,  2769,   438,   322, 18926,   432,   971, 23571,    49,
           203,    20, 35462,  2150,    20,    47, 10246,  1018, 21147,  4998,
           301,   343,   839,    44,    60, 23571, 36855,    44, 21147,  1018,
         21147,   638,   320,  2831,   203,    20,  4594,    20,    47,  2769,
           438,   322, 18926,   432, 15070,    49,   203,    20, 35462,  2150,
            20,    47, 10246,  1018, 21147,  4998,   301,   343,   839,    44,
         32713, 36855,    44, 21147,  

In [3]:
question = "What is the capital of France?\n"
sparql_query = "SELECT ?capital WHERE { dbr:France dbo:capital ?capital .}"

# check if the tokenizer automatically adds the necessary special tokens and attention mask.
inputs = tokenizer.encode_plus(question + sparql_query, return_tensors="pt")
inputs

{'input_ids': tensor([[ 8197,   438,   322, 18926,   432, 45600,    49,   203,  4620,  1018,
         21147,  4998,   301,   343,   839,    44,  6082,   723, 36855,    44,
         21147,  1018, 21147,   638,   111]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [4]:
# manually adding eos_token
question = "What is the capital of France?\n<|endoftext|>"
sparql_query = "SELECT ?capital WHERE { dbr:France dbo:capital ?capital .}"

inputs = tokenizer.encode_plus(question + sparql_query, return_tensors="pt")
inputs

{'input_ids': tensor([[ 8197,   438,   322, 18926,   432, 45600,    49,   203,     0,  4620,
          1018, 21147,  4998,   301,   343,   839,    44,  6082,   723, 36855,
            44, 21147,  1018, 21147,   638,   111]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [7]:
# check how tokenizer is dealing with the spaces  around "?" and "."

sparql_query = "SELECT ?capital WHERE { dbr:France dbo:capital ?capital .}"

inputs = tokenizer.encode_plus(sparql_query, return_tensors="pt")
print(tokenizer.decode(inputs['input_ids'][0]))

SELECT?capital WHERE { dbr:France dbo:capital?capital.}


In [12]:
# Fixing mis-tokenized spaces around "?" and "." tokens

sparql_query = "SELECT ?capital WHERE { dbr:France dbo:capital ?capital .}"

inputs = tokenizer.encode_plus(sparql_query, return_tensors="pt")
print('Raw query tokenization => ', tokenizer.decode(inputs['input_ids'][0]))

sparql_query = sparql_query.replace("?", " ?").replace(".", " . ")
inputs = tokenizer.encode_plus(sparql_query, return_tensors="pt")
print('Pre-processesd query tokenization => ', tokenizer.decode(inputs['input_ids'][0]))

Raw query tokenization =>  SELECT?capital WHERE { dbr:France dbo:capital?capital.}
Pre-processesd query tokenization =>  SELECT ?capital WHERE { dbr:France dbo:capital ?capital . }


In [46]:
question = "What is the capital of France?"
sparql_query = "SELECT ?capital WHERE { dbr:France dbo:capital ?capital .}"

# manually adding eos_token
inputs = tokenizer.encode_plus(question + eos_token + sparql_query, return_tensors="pt")
inputs

{'input_ids': tensor([[ 8197,   438,   322, 18926,   432, 45600,    49,     0,  4620,  1018,
         21147,  4998,   301,   343,   839,    44,  6082,   723, 36855,    44,
         21147,  1018, 21147,   638,   111]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [55]:
# adding EOS token between pairs of questions and answers 

prompt = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=50, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"SPARQL"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }<|endoftext|>

"question"= what is the capital of USA?
"SPARQL"= SELECT?capital WHERE { dbr:USA dbo:capital?capital. }<|endoftext|>

"question"= what is the capital of Italy?
"SPARQL"= SELECT?capital WHERE { dbr:Italy dbo:capital?capital. }
<|endoftext|>


In [56]:
# adding EOS token between pairs of questions and answers 

prompt = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"SPARQL"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }<|endoftext|>

"question"= what is the capital of USA?
"SPARQL"= SELECT?capital WHERE { dbr:USA dbo:capital?capital. }<|endoftext|>

"question"= what is the capital of Italy?
"SPARQL"= SELECT?capital WHERE { dbr:Italy dbo:capital?capital. }
<|endoftext|>


In [59]:
# adding EOS token between pairs of questions and answers 

prompt_without_eos = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }

"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }

"question"= what is the capital of Italy?
"""

prompt_with_eos = """
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"SPARQL"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of USA?
"SPARQL"= SELECT ?capital WHERE { dbr:USA dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"""


inputs = tokenizer.encode_plus(prompt_without_eos, return_tensors="pt").to(device)
outputs = model.generate(
    inputs['input_ids'],
    max_new_tokens=50,
    attention_mask=inputs['attention_mask'])
print('Before adding EOS:\n', tokenizer.decode(outputs[0]))

print("="*35)

inputs = tokenizer.encode_plus(prompt_with_eos, return_tensors="pt").to(device)
outputs = model.generate(
    inputs['input_ids'],
    max_new_tokens=100,
    attention_mask=inputs['attention_mask'])
print('After adding EOS:\n', tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Before adding EOS:
 
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

"question"= what is the capital of Japan?
"SPARQL"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }

"question"= what is the capital of USA?
"SPARQL"= SELECT?capital WHERE { dbr:USA dbo:capital?capital. }

"question"= what is the capital of Italy?
"SPARQL"= SELECT?capital WHERE { dbr:Italy dbo:capital?capital. }

"question"= what is the capital of France?
"SPARQL"= SELECT?capital WHERE { dbr:Fr
After adding EOS:
 
Compelete "SPARQL" by translating "question" into a SPARQL query usig DBpedia prefixes below:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"SPARQL"= SELECT?capital WHERE { dbr:Japan dbo:capital?capital. }<|endoftext|>

"question"= what is the capital of USA?
"SPARQL"= SELECT

#
# Falcon 1B

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


checkpoint = "tiiuae/falcon-rw-1b"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

Loading tiiuae/falcon-rw-1b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-rw-1b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading tiiuae/falcon-rw-1b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-rw-1b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading tiiuae/falcon-rw-1b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-rw-1b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'FalconForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Pegasu

In [4]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}
"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}
}
}
This output is from a SQL query on DBpedia’s ontology of Germany. The first query shows the capital as Berlin. The second query shows the capital as Frankfurt. These two examples show that the German capital and the Chinese capital are not the same. However, they are both correct.
The German capital, Berlin, is the correct German capital; therefore, i

In [5]:
%%time

prompt = """
input= SPARQL Query What is the capital of France?

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query What is the capital of France?


input= SPARQL Query What is the value of the year?
What is a good example of an RDF graph?
The best way to learn how to use RDF is to start with an RDF Graph. In this example, we will create a simple SPARQL query that will return the value of the year. This graph is an RDF graph.
How do you use graphviz to create a RDF graph?
Here is a simple Graph.
How do you create an RDF graph in R?
To create a RDF graph, you need two files: a source file for the data (a list of values or objects), and a target file for the RDF graph. This target file must be a text document. The following example shows how to do that.
What are RDF graphs?
In a nutshell, RDF graphs are a representation of data. This can be a data set, a data structure (e.g. a relational table), or an RDF triplestructure.
How does RDF work?
RDF (Resource Description Framework) is an OWL-based standard that aims to provide a common vocabulary to describe web pages, document

In [6]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
out

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
out


input= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
?capital {
 }
}
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
!?capital }
}
input= PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
?capital {
 }
}
Output from the query above:
<?p="{?capital}"?>?capital
The query outputs: <?p="{?capital}"?>?capital
The first element of the?p array is the value for the?capital field of the resource.
The first element o

In [7]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?


input= SPARQL Query For what is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
  dbr:China dbo:capital?capital.
}
}
The capital of Germany is Berlin, and the capital of Iran is Teheran.
- Posted using Chat Catcher
- Login to post comments
-
CPU times: user 1.38 s, sys: 0 ns, total: 1.38 s
Wall time: 1.38 s


In [8]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

 dbr:Japan dbo:capital?capital.
}
}
CPU times: user 204 ms, sys: 0 ns, total: 204 ms
Wall time: 203 ms


In [9]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Belgium?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Belgium?
output=


output-
output-
output-
Output 1: the capital of Belgium
output-
output-
output=
output=
output=
Output 2: the capital of Belgium in a different format
output-
output=
output-
output=
output=
Output 3: a different format for the capital of Belgium
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
Output 4: what is the capital of France?
output=
output=
output=
input=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
input= SPARQL Query for what is the capital of France?
output=
output=
Output 5: the capital of France in a different format
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=


In [10]:

prompt = """
input= SPARQL Query for what is the population of Japan?
output=
"""

sequences = pipeline(
    prompt,
    max_length=100,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the population of Japan?
output=
> SELECT *
> FROM population WHERE (
> (
> input= “Japan” or input= “Japan” )
> or
> (
> input= “Japan” or input= “Japan” )
> and
> (
> input= "Japan" or input= "Japan") or
> input= "Japan"


In [11]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

"""

sequences = pipeline(
    prompt,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=




In [12]:
tokenizer.eos_token_id


50256

In [13]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

"""

sequences = pipeline(
    prompt,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=



input= SPARQL Query for what is the population of Germany?
output= ?population 
WHERE {
?population 
}

input= SPARQL Query for what is the capital of Austria?
output= 
}

output= SELECT?capital 
WHERE {
?capital {
?capital {
?capital {
?population 
}
}
}
}
input= SPA

In [14]:
len(tokenizer(prompt)['input_ids'])

245

In [15]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

"""

sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

 dba:France dbr:Population 
WHERE {
  dbr:Germany dbo:population ?population.
}

input= SPARQL Query for what is the population of France?
output= PREFIX dbo: <http://


In [16]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output=
"""

sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
input= SPARQL Query for what is the capital of Japan?
output=
I have to write code in PHP?


#
# CodeGen-350M-multi


In [17]:
checkpoint = "Salesforce/codegen-350M-multi"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

In [18]:
prompt = "SPARQL Query for: what is the capital of Japan?"

sequences = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: SPARQL Query for: what is the capital of Japan?
 *
 * @author  Benjamin Bengfort <benjamin@bengfort.com>
 * @author  Ricardo Cerqueira - 2015
 * @version $Id$
 *
 * @par
 * $Id$
 */
//------------------------------------------------------------------------------
// Copyright (C) 2013, The Regents of the University of California
// See README in root directory for license.
//------------------------------------------------------------------------------

#include "query.h"

//------------------------------------------------------------------------------
// Private methods
//------------------------------------------------------------------------------

/**
 * Return the capital of Japan.
 */
Query::Value Query::Capitals( const QString& query ) const
{
    // Build the Query object for the query
    Query q( query );

    // Create the JSON object
    q.SetJSON( true );

    // Retrieve the results
    QVariant


In [19]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
 PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELEC

In [20]:

prompt = """
input= SPARQL Query for what is the capital of Thailand?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Thailand?
output=




In [ ]:
checkpoint = "EleutherAI/gpt-neo-2.7B"

#
# GPT-NEO-2.7B


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


checkpoint = "EleutherAI/gpt-neo-2.7B"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:
prompt = "SPARQL Query for: what is the capital of Japan?"

sequences = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: SPARQL Query for: what is the capital of Japan?

I'm writing a program to search an RDF database and I need a way to get the capital of Japan. I don't know anything about RDF, but I know the following:

Japan is a country, which has a capital, which is Tokyo.
A country is made of a number of provinces, which is Japan's largest, so I would expect to find a query for "Japan", but not "Japan, the capital of".

So, I'm wondering if there is a way to search for "Japan, the capital of", in an RDF database (assuming the RDF database is in a JSON format) and if it exists, whether I should write my own query for it, or use a query that already exists?

A:

The query you are looking for is "Japan, the capital of". However, you should use the URI of Tokyo in your


In [4]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
"""

sequences = pipeline(
    prompt,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
  dbr:Norway dbo:capital?

In [5]:

prompt = """
input= SPARQL Query for what is the capital of Thailand?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Thailand?
output=

  <http://dbpedia.org/sparql>
  <http://dbpedia.org/sparql>

where <http://dbpedia.org/sparql> is the SPARQL endpoint of DBpedia, and


input: a SPARQL query
output:  the result of this query in the form of a graph


input= SPARQL Query for what is the capital of Thailand?
output=

  <http://dbpedia.org/sparql>
  <http://dbpedia.org/sparql>

where <http://dbpedia.org/sparql> is the SPARQL endpoint of DBpedia, and


output: the result of this query in the form of a graph


output=

  <http://dbpedia.org/sparql>

where <http://dbpedia.org/sparql> is the SPARQL endpoint of DBpedia

*/



In [8]:

prompt = """
input= SPARQL Query for what is the capital of Thailand?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Thailand?
output=



# The query below uses the same input and returns the same result as the
# query above.

SPARQL =





# The above query uses the "SPARQL" function to query SPARQL and returns
# the value "Thailand" in output.

spqr = spqr { 		
    "?sparql"   			
    "?sparql" 		
}




# The above query uses the "SPARQL" function to query SPARQL and returns
# null in output.

spqr null




    </textarea>
  </div>
  <script>
    var editor = CodeMirror.fromTextArea(document.getElementById("code2"), {
      mode: "text/x-sparql"
    });
  </script>

  <p><strong>MIME types defined:</strong> <code>application/sparql-query</code>.</p>

  </article>



In [11]:

prompt = """
input= Capital of Japan
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= Capital of United States
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:United_States dbo:capital ?capital .
}

input= Capital of China
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= Capital of France
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:France dbo:capital ?capital .
}

# Capital of Germany
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= Capital of Canada
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Canada dbo:capital ?capital .
}

input= Capital of Australia
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Australia dbo:capital ?capital .
}

input= Capital of India
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:India dbo:capital ?capital .
}

input= Capital of Russia
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Russia dbo:capital ?capital .
}

input= Capital of Spain
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= Capital of Brazil
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Brazil dbo:capital ?capital .
}

# Capital of South Africa
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:South_Africa dbo:capital ?capital .
}

input= Capital of Italy
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Italy dbo:capital ?capital .
}

input= Capital of Mexico
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Mexico dbo:capital ?capital .
}

input= Capital of Argentina
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Argentina dbo:capital ?capital .
}

input= Capital of Turkey
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Turkey dbo:capital ?capital .
}

input= Capital of Iran
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}

input= Capital of Egypt
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Egypt dbo:capital ?capital .
}

input= Capital of Indonesia
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Indonesia dbo:capital ?capital .
}

input= Capital of Sweden


"""

sequences = pipeline(
    prompt,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1254, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Result: 
input= Capital of Japan
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= Capital of United States
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:United_States dbo:capital ?capital .
}

input= Capital of China
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= Capital of France
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:France dbo:capital ?capital .
}

# Capital of Germany
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= Capital of Canada
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/

In [13]:

prompt = """
input= Capital of Sweden

"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= Capital of Sweden




In [15]:

prompt = """
Write an example fo sparql query over dbpedia
"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
Write an example fo sparql query over dbpedia

I am new to sparqler. I am writing some example query in sparql to fetch the result of a query, and I am not able to get the exact output as given


In [16]:

prompt = """

Translate the following sentence into a SPARQL query: find capital of Japan

"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Translate the following sentence into a SPARQL query: find capital of Japan

    SELECT?c?o WHERE{
   ?c <http://purl.org/dc/elements/1.


In [22]:

prompt = """

Translate the following sentence into a SPARQL query for DBpedia: find capital of Japan

"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Translate the following sentence into a SPARQL query for DBpedia: find capital of Japan



?capital.name

?city.name

?population.n

?population.i

?population


In [20]:

prompt = """

Fill in the SPARQL query template to find the capital of a specific country. 
The country's name is 'Germany'.
The template is: 
'SELECT ?capital WHERE { dbr:" " dbo:capital ?capital . }'

"""

sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Fill in the SPARQL query template to find the capital of a specific country. 
The country's name is 'Germany'.
The template is: 
'SELECT ?capital WHERE { dbr:" " dbo:capital ?capital . }'

    SELECT * WHERE {

                         ?capital a geo:Point ;
                                                                 <-- 
    dbo:capital geo:Point.
                          }

This query will return 'Frankfurt'
I've been trying to use the SPARQL function ST_DWithin in order to find the capital of a specific country. 
In the following code, I've created a function to create an RDF query. 

#include <iostream>
#include <string>
#include <sparql/sparql_execution_context.h>
#include <spar


In [25]:

prompt = """

Translate the following question into SPARQL query towards DBpedia knowledge base using these prefixes:

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

Q: What is the capital of Japan
A:

"""

sequences = pipeline(
    prompt,
    max_length=00,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Translate the following question into SPARQL query towards DBpedia knowledge base using these prefixes:

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

Q: What is the capital of Japan
A:

                                                                                                                                                                                                                                    
